In [1]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

%cd ../..

from app.users.models import User
from app.db import get_session
from cassandra.cqlengine.management import sync_table

get_session()
sync_table(User)

C:\Users\DELL\devworkspace\practice\video_membership


c:\users\dell\devworkspace\practice\video_membership\venv\lib\site-packages\cassandra\cqlengine\management.py:545: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [2]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [3]:
post_data = {
    'email': 'udo@gmail.com',
    "password": 123
}

In [4]:
UserLoginSchema(**post_data)
list(User.objects.all())

[User(email = bobs@gmail.com, user_id = 5ba800a0-3955-11ee-8824-c0389681892e),
 User(email = kingscode125@gmail.com, user_id = 617b792e-3b9c-11ee-993c-c0389681892e),
 User(email = kingsley@gmail.com, user_id = 6a229968-394b-11ee-a85b-c0389681892e),
 User(email = kingsley@gmail.com, user_id = cb76fccd-3952-11ee-b137-c0389681892e),
 User(email = kingsley@gmail.com, user_id = 41b2a3c9-3953-11ee-9caa-c0389681892e),
 User(email = bob@gmail.com, user_id = 4811c407-3955-11ee-b9e2-c0389681892e),
 User(email = kingscode124@gmail.com, user_id = 6a229968-394b-11ee-a85b-c0389681892e),
 User(email = kingscode124@gmail.com, user_id = 6b7e4e77-3953-11ee-81f9-c0389681892e),
 User(email = kingscode125@goat.com, user_id = f63b511d-3b9c-11ee-8ba4-c0389681892e)]

In [5]:
class UserSignUpSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr

    @validator('email')
    def validate_email_exists(cls, v, values, **kwargs):
        email = v
        queryset = User.objects.filter(email= email)
        if queryset.count() != 0:
            raise ValueError("Email is not avaliable.....try using another email")
        return email

    @validator('password_confirm')
    def password_match(cls, v, values, **kwargs):
        password = values.get("password")
        password_confirm = v
        print(password, password_confirm)

        if password != password_confirm:
            raise ValueError("Passwords do not match")
        return v

In [6]:
data = UserSignUpSchema(email = 'abc@gmail.com', password = 'abc', password_confirm = 'abc')

********** **********


In [7]:
# lambda x: for i in x
data.dict()['password'].get_secret_value()

'abc'